# OWL 赛事数据统计

根据 liquipedia 数据统计 OWL 解说以及和比赛比分等信息

### 基础模板解析

In [149]:
from datetime import datetime, timedelta
import re
# import wikitextparser as wtp
# https://github.com/5j9/wikitextparser#miscellaneous
from collections import defaultdict
from copy import copy

In [124]:
datetime.strptime('May 5, 2022 - 12:00', '%B %d, %Y - %H:%M') - timedelta(hours=-15)

datetime.datetime(2022, 5, 6, 3, 0)

In [125]:
from datetime import datetime, timedelta
files = ['liquipedia/KickoffClash_Qualifiers.txt',
         'liquipedia/KickoffClash_Tournament.txt',
         'liquipedia/MidseasonMadness_Qualifiers.txt',
         'liquipedia/MidseasonMadness_Tournament.txt',
         'liquipedia/SummerShowdown_Qualifiers.txt',
         'liquipedia/SummerShowdown_Tournament.txt']
# 寻找所有用到的时区代号
timezones = set()
for file in files:
    with open(file) as f:
        data = f.readlines()
    data = ''.join(data)
    timezones = set.union(timezones, set(re.findall('{{Abbr/(.+?)}}', data)))
print(sorted(timezones))
#        {'BST', 'CDT', 'CEST', 'CST', 'EDT', 'KST', 'PDT', 'UTC'}
deltas = [   +1,    -5,     +2,    +8,    -4,    +9,    -7,     0]
# （美国)东部夏令时, 英国夏时制, （美国)太平洋夏令时, 世界时, 韩国标准时间. 欧洲中部夏令时间, 北京时间, （美国)中部夏令时
# https://blog.csdn.net/whatday/article/details/109856495

tdelta = dict([(k, timedelta(hours=deltas[i]-8)) for i, k in enumerate(sorted(timezones))])
tdelta
# 为了统一修正到北京时间, 需要将 liquipedia 中的时间 - delta

['BST', 'CDT', 'CEST', 'CST', 'EDT', 'KST', 'PDT', 'UTC']


{'BST': datetime.timedelta(days=-1, seconds=61200),
 'CDT': datetime.timedelta(days=-1, seconds=39600),
 'CEST': datetime.timedelta(days=-1, seconds=64800),
 'CST': datetime.timedelta(0),
 'EDT': datetime.timedelta(days=-1, seconds=43200),
 'KST': datetime.timedelta(seconds=3600),
 'PDT': datetime.timedelta(days=-1, seconds=32400),
 'UTC': datetime.timedelta(days=-1, seconds=57600)}

In [126]:
s_liq1 = '''|M3={{Match
    |date=September 2, 2022 - 12:00 {{Abbr/PDT}}
    |youtube=overwatchleague
    |opponent1={{TeamOpponent|atl}}
    |opponent2={{TeamOpponent|gla}}
    |map1={{Map|map=Lijiang Tower|mode=Control|score1=1|score2=2|winner=2}}
    |map2={{Map|map=Hollywood|mode=Hybrid|score1=2|score2=3|winner=2}}
    |map3={{Map|map=Dorado|mode=Escort|score1=0|score2=1|winner=2}}
    |map4={{Map|map=Colosseo|mode=Push|score1=|score2=|winner=Skip}}
    |map5={{Map|map=Nepal|mode=Control|score1=|score2=|winner=Skip}}
	|mvp=Happy
	|comment=
	|owl=39227
	|overgg=
	|vod=
}}
'''
# w = wtp.parse(re.sub('\n( |\t)+', '', s_liq))
# print(w.templates[4].arguments)

In [127]:
def wiki2dict(s):
    d = defaultdict(str)
    sd = s.replace('{', '') \
        .replace('}', '') \
        .split('|')[1:]
    dl  = [i           for i in sd if '=' in i ]
    idx = [i.find('=') for i in sd if '=' in i ]
    for sdi, i in zip(dl, idx):
        d[sdi[:i]] = sdi[i+1:].strip()
    return d

def parse_wiki(s_liq, is_name=True):
    # refactoring the code
    s_wiki = re.sub('\n\s+', '', s_liq).strip()
    s_wiki = s_wiki.replace('opponent', 'team')# compatible with former data
    tz = re.findall('\{\{Abbr/(.+?)\}\}', s_wiki)[0]
    s_wiki = s_wiki.replace('{{Abbr/' + tz + '}}', '')
    if is_name:
        i = s_wiki.find("=")
        name = s_wiki[1:i]
        # team1, team2, map1-5, comment1-2
        s_list = re.findall('\{\{.+?\}\}', s_wiki[i+3:])
        s = re.sub('\{\{.+?\}\}', '', s_wiki[i+3:])
    else:
        name = ''
        s_list = re.findall('\{\{.+?\}\}', s_wiki)
        s = re.sub('\{\{.+?\}\}', '', s_wiki)   

    # get_name = lambda s: re.search(r'\{\{.+\|(.+)\|.+\}\}', s).group(1)
    get_name = lambda s: s.replace('{', '').replace('}', '').split('|')[1]

    d = wiki2dict(s)
    # bestof
    if 'bestof' not in s_wiki:
        d['bestof'] = max([int(i) for i in re.findall('\|map([1-9])', s_liq1)])
    # name
    d['name'] = name
    # date
    d['date'] = datetime.strptime(d['date'].strip(), '%B %d, %Y - %H:%M')   # June 2, 2022 - 19:00 {{Abbr/UTC}}
    d['date'] = d['date'] - tdelta[tz]
    # opponent / team 
    for i in [1, 2]:
        d[f'team{i}'] = get_name(s_list[i-1])
    # comment
    if 'comment' in s_wiki:
        if not re.search('\|comment=\|', s_wiki):
            for i in [1, 2]:
                d[f'comment{i}' ] = get_name(s_list[i - 3])
        del d['comment']
    # flatten every map dict
    # print(d['bestof'])
    for i in range(int(d['bestof'])):
        del d[f'map{i+1}']
        dmap = wiki2dict(s_list[i-1 + 3])
        for kmap in dmap.keys():
            v = dmap[kmap]
            if 'score' in kmap or 'winner' in kmap:
                if len(v) == 1:
                    v = int(v)
                elif v.strip().lower() == 'skip':
                    v = ''
                elif v.strip().lower() == 'draw':
                    v = -1
                elif len(v) != 0:
                    v = float(v) if v[-1].isdigit() else float(v[:-1])
            d[f'{kmap}_map{i+1}'] = v
    return d
        
parse_wiki(s_liq1)

defaultdict(str,
            {'date': datetime.datetime(2022, 9, 3, 3, 0),
             'youtube': 'overwatchleague',
             'team1': 'atl',
             'team2': 'gla',
             'mvp': 'Happy',
             'owl': '39227',
             'overgg': '',
             'vod': '',
             'bestof': 5,
             'name': 'M3',
             'map_map1': 'Lijiang Tower',
             'mode_map1': 'Control',
             'score1_map1': 1,
             'score2_map1': 2,
             'winner_map1': 2,
             'map_map2': 'Hollywood',
             'mode_map2': 'Hybrid',
             'score1_map2': 2,
             'score2_map2': 3,
             'winner_map2': 2,
             'map_map3': 'Dorado',
             'mode_map3': 'Escort',
             'score1_map3': 0,
             'score2_map3': 1,
             'winner_map3': 2,
             'map_map4': 'Colosseo',
             'mode_map4': 'Push',
             'score1_map4': '',
             'score2_map4': '',
             'winner_map

In [128]:
# the data format is quiet different in `liquipedia/KickoffClash_Qualifiers.txt`
s_liq2 = '''{{MatchMaps
|date=May 5, 2022 - West
|team1=nye|games1=1
|team2=gla|games2=3
|winner=2
|details={{BracketMatchSummary
|date=May 5, 2022 - 12:00 {{Abbr/PDT}}
|youtube=overwatchleague
|finished=true
|comment=Casters: {{player|Uber|flag=au}} & {{player|Mr X|flag=us}}
|map1=Lijiang Tower|map1score=1-2|map1win=2
|map2=Midtown|map2score=0-2|map2win=2
|map3=Circuit Royal|map3score=3-2|map3win=1
|map4=New Queen Street|map4score=71.89-103.57|map4win=2
|map5=Ilios|map5score=|map5win=skip
|vodgame1=https://youtu.be/4HC6Ev_dlyI?t=234
|vodgame2=https://youtu.be/4HC6Ev_dlyI?t=1613
|vodgame3=https://youtu.be/4HC6Ev_dlyI?t=3233
|vodgame4=https://youtu.be/4HC6Ev_dlyI?t=5011
|mvp=Kevster
|owl=38971
|overgg=15902
|pf=
|wl=
}}
'''

liquipedia/KickoffClash_Qualifiers.txt 文件中存储比赛的格式用的和剩下的还都不相同, 需要额外标准化处理

In [129]:
def fix(s_liq2):
    s = re.sub(r'\|team([0-9])=(.+?)\|\S+\n',
            r'|opponent\1={{TeamOpponent|\2|score=}}\n',
            s_liq2.replace('|details={{BracketMatchSummary\n', ''))
    s = re.sub('\{\{MatchMaps\n.+\n\|team',
            '{{Match\n|bestof=5\n',
            s)
    s = re.sub('\|comment=(.+)\n((\s|\S)+)\n\}\}',
            r'\2\n|comment=\1\n}}',
                s)
        
    maps_type = ['Control', 'Hybrid', 'Escort', 'Push', 'Control']
    for i in range(1, 6):
        pattern = f'\|map{i}=(.+)?\|map{i}score=(.+)?\|map{i}win=(.+)?'
        m = re.findall(pattern, s)[0]
        if '-' in m[1]:
            s1, s2 = m[1].split('-')
            rep = f"|map{i}={m[0]}|map{i}score={s1}-{s2}|map{i}win={m[2]}"
        else:
            s1, s2 = '', ''
            rep = f"|map{i}={m[0]}|map{i}score={m[1]}|map{i}win={m[2]}"
        after = f'|map{i}={{{{Map|map={m[0]}|mode={maps_type[i-1]}|score1={s1}|score2={s2}|winner={m[2]}}}}}'
        s = s.replace(rep, after)
    return s

parse_wiki(fix(s_liq2), is_name=False)

defaultdict(str,
            {'date': datetime.datetime(2022, 5, 6, 3, 0),
             'team1': 'nye',
             'team2': 'gla',
             'winner': '2',
             'youtube': 'overwatchleague',
             'finished': 'true',
             'vodgame1': 'https://youtu.be/4HC6Ev_dlyI?t=234',
             'vodgame2': 'https://youtu.be/4HC6Ev_dlyI?t=1613',
             'vodgame3': 'https://youtu.be/4HC6Ev_dlyI?t=3233',
             'vodgame4': 'https://youtu.be/4HC6Ev_dlyI?t=5011',
             'mvp': 'Kevster',
             'owl': '38971',
             'overgg': '15902',
             'pf': '',
             'wl': '',
             'bestof': 5,
             'name': '',
             'comment1': 'Uber',
             'comment2': 'Mr X',
             'map_map1': 'Lijiang Tower',
             'mode_map1': 'Control',
             'score1_map1': 1,
             'score2_map1': 2,
             'winner_map1': 2,
             'map_map2': 'Midtown',
             'mode_map2': 'Hybrid',
         

### 整理汇总各阶段数据

In [130]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
files = ['liquipedia/KickoffClash_Qualifiers.txt',      # 37 / 60 comment data given, fixed manually
         'liquipedia/KickoffClash_Tournament.txt',      # R2M2, R4M2, R5M1(Grand Final) data in file on liquipedia is not standard, fixed manually
         'liquipedia/MidseasonMadness_Qualifiers.txt',
         'liquipedia/MidseasonMadness_Tournament.txt',
         'liquipedia/SummerShowdown_Qualifiers.txt',
         'liquipedia/SummerShowdown_Tournament.txt']
keys = ['bestof', 'date', 'team1', 'team2', 'owl', 'overgg', 'mvp', 'vod', 'comment1', 'comment2',
        's1m1', 's2m1', 'wm1',
        's1m2', 's2m2', 'wm2',
        's1m3', 's2m3', 'wm3',
        's1m4', 's2m4', 'wm4',
        's1m5', 's2m5', 'wm5',
        's1m6', 's2m6', 'wm6',
        's1m7', 's2m7', 'wm7']      # s1m3 含义为 score of team1 in map3, wm3 含义为 winner in map3

#### 测试各文件中数据是否正常获得

In [131]:
with open('liquipedia/KickoffClash_Qualifiers.txt') as f:
    data = f.readlines()
data = ''.join(data)
matches = re.findall('\{\{MatchMaps\n[\s\S]*?\n\}\}\n', data)
print(len(matches))
for m in matches:
    # print(m, len(m))
    d = parse_wiki(fix(m), is_name=False)
    for k in keys:
        d[k]

60


In [132]:
# file = 'liquipedia/KickoffClash_Tournament.txt'
# file = 'liquipedia/MidseasonMadness_Qualifiers.txt'
# file = 'liquipedia/MidseasonMadness_Tournament.txt'
file = 'liquipedia/SummerShowdown_Qualifiers.txt'
# file = 'liquipedia/SummerShowdown_Tournament.txt'
with open(file) as f:
    data = f.readlines()
data = ''.join(data)
matches = re.findall('\|.+\{\{[\s\S]*?\n\}\}\n', data)
print(len(matches))
for m in matches:
    d = parse_wiki(m)
    for k in keys:
        d[k]

60


#### 汇总

In [275]:
N = len(keys)
lb, ld, lt1, lt2, lo, lgg, lmvp, lv, lc1, lc2, lwm1, lwm2, lwm3, lwm4, lwm5, lwm6, lwm7, \
ls1m1, ls1m2, ls1m3, ls1m4, ls1m5, ls1m6, ls1m7, ls2m1, ls2m2, ls2m3, ls2m4, ls2m5, ls2m6, ls2m7 = [list() for _ in range(N)]
lwm  = [lwm1, lwm2, lwm3, lwm4, lwm5, lwm6, lwm7]
ls1m = [ls1m1, ls1m2, ls1m3, ls1m4, ls1m5, ls1m6, ls1m7]
ls2m = [ls2m1, ls2m2, ls2m3, ls2m4, ls2m5, ls2m6, ls2m7]
ls1, ls2, lm = [list() for _ in range(3)]
with open('liquipedia/KickoffClash_Qualifiers.txt') as f:
    data = f.readlines()
data = ''.join(data)
matches = re.findall('\{\{MatchMaps\n[\s\S]*?\n\}\}\n', data)

prefix = 'liquipedia'
files = ['KickoffClash_Tournament.txt', 'MidseasonMadness_Qualifiers.txt', 'MidseasonMadness_Tournament.txt', 'SummerShowdown_Qualifiers.txt', 'SummerShowdown_Tournament.txt']
for file in files:
    with open(f'{prefix}/{file}') as f:
        data = f.readlines()
    data = ''.join(data)
    matches += re.findall('\|.+\{\{[\s\S]*?\n\}\}\n', data)
    
for i, m in enumerate(matches):
    if i < 60:      # First 60 matches from KickoffClash_Qualifiers.txt
        d = parse_wiki(fix(m), is_name=False)
    else:
        d = parse_wiki(m)
    if len(d['mvp']) != 0:
        lb.append(d['bestof'])
        ld.append(d['date'])
        lt1.append(d['team1'])
        lt2.append(d['team2'])
        lo.append(d['owl'])
        lgg.append(d['overgg'])
        lmvp.append(d['mvp'])
        lv.append(d['vod'])
        lc1.append(d['comment1'])
        lc2.append(d['comment2'])
        for i in range(7):
            lwm[i].append(d[f'winner_map{i+1}'])
            ls1m[i].append(d[f'score1_map{i+1}'])
            ls2m[i].append(d[f'score2_map{i+1}'])

        # lwm1.append(d['winner_map1'])
        # lwm2.append(d['winner_map2'])
        # lwm3.append(d['winner_map3'])
        # lwm4.append(d['winner_map4'])
        # lwm5.append(d['winner_map5'])
        # lwm6.append(d['winner_map6'])
        # lwm7.append(d['winner_map7'])
        # count score and total map
        ls1.append(np.sum([1 for i in range(7) if d[f'winner_map{i}'] == 1]))
        ls2.append(np.sum([1 for i in range(7) if d[f'winner_map{i}'] == 2]))
        lm.append(np.sum([1 for i in range(7) if d[f'winner_map{i}'] != '']))
# create data frame
df = pd.DataFrame(np.c_[lb, ld, lt1, lt2, lo, lgg, lmvp, lv, lc1, lc2, \
                        ls1m1, ls2m1, lwm1, \
                        ls1m2, ls2m2, lwm2, \
                        ls1m3, ls2m3, lwm3, \
                        ls1m4, ls2m4, lwm4, \
                        ls1m5, ls2m5, lwm5, \
                        ls1m6, ls2m6, lwm6, \
                        ls1m7, ls2m7, lwm7, ls1, ls2, lm], 
                        columns=keys + ['score1', 'score2', 'maps'])
df = df.sort_values('date', ascending=False)
df

,bestof,date,team1,team2,owl,overgg,mvp,vod,comment1,comment2,...,wm5,s1m6,s2m6,wm6,s1m7,s2m7,wm7,score1,score2,maps
232,5,2022-09-10 19:25:00,shd,seo,39283,16474,LIP,,,,...,,,,,,,,3,0,3
233,5,2022-09-10 18:00:00,gzc,phi,39286,16475,MN3,,,,...,,,,,,,,0,3,3
228,5,2022-09-10 07:00:00,was,tor,39271,16472,ALTHOUGH,,,,...,,,,,,,,1,3,4
229,5,2022-09-10 05:35:00,van,fla,39274,16473,Checkmate,,,,...,,,,,,,,0,3,3
227,5,2022-09-10 03:45:00,sfs,hou,39273,16471,Proper,https://www.youtube.com/watch?v=mpg9bcY9ggA,,,...,,,,,,,,3,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,5,2022-05-07 04:30:00,bos,van,38974,15907,Punk,,LemonKiwi,LEGDAY,...,1,,,,,,,3,2,5
3,5,2022-05-07 03:00:00,ldn,sfs,38975,15908,s9mm,,Jaws,Nekkra,...,,,,,,,,0,3,3
2,5,2022-05-06 06:00:00,fla,atl,38973,15904,UltraViolet,,Jaws,Nekkra,...,,,,,,,,1,3,4
1,5,2022-05-06 04:30:00,sfs,par,38972,15903,FiNN,,Uber,Mr X,...,,,,,,,,3,0,3


#### 和以前结果合并

In [163]:
# 队伍代码和队名
dteam = {'atl': '亚特兰大君临队',
 'bos': '波士顿崛起队',
 'cdh': '成都猎人队',
 'dal': '达拉斯燃料队',
 'fla': '佛罗里达狂欢队',
 'gla': '洛杉矶角斗士队',
 'gzc': '广州冲锋队',
 'hou': '休斯敦神枪手队',
 'hzs': '杭州闪电队',
 'ldn': '伦敦喷火战斗机队',
 'nye': '纽约九霄天擎队',
 'par': '巴黎永生队',
 'phi': '费城融合队',
 'seo': '首尔王朝队',
 'sfs': '旧金山震动队',
 'shd': '上海龙之队',
 'tor': '多伦多捍卫者队',
 'val': '洛杉矶英勇队',
 'van': '温哥华泰坦队',
 'was': '华盛顿正义队'}

##### 和 B 站 api 获取数据对比

In [103]:
df_old = pd.read_excel('owl_data.xlsx', usecols=range(1, 13))       # 有的队名标题把 '休斯敦神枪手队' 打成了 '休斯顿神枪手队', 都替换为了前者
df_old

,month,day,play,team1,team2,length,bvid,name1,name2,score1,score2,maps
0,8,20,7935,洛杉矶英勇队,费城融合队,117.600000,BV1eT411c71L,狂疯小逗,老陈,2,3,5
1,8,20,9753,上海龙之队,杭州闪电队,70.350000,BV1rd4y1w72n,九朵,老陈,3,0,3
2,8,20,15333,首尔王朝队,成都猎人队,82.700000,BV1qU4y1r7M1,九朵,狂疯小逗,3,0,3
3,8,20,8725,多伦多捍卫者队,达拉斯燃料队,85.450000,BV1rT411c7db,娓娓,杨小龙,1,3,4
4,8,20,5075,旧金山震动队,波士顿崛起队,57.116667,BV13P411j7as,杨小龙,时光,3,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
178,5,7,35572,波士顿崛起队,温哥华泰坦队,117.050000,BV1KA4y1S7Jh,九朵,Alan,3,2,5
179,5,7,36078,伦敦喷火战斗机队,旧金山震动队,65.916667,BV1GA4y1S76n,九朵,时光,0,3,3
180,5,6,68021,佛罗里达狂欢队,亚特兰大君临队,83.733333,BV1XL4y1c74j,木子,赤小兔,1,3,4
181,5,6,53709,旧金山震动队,巴黎永生队,56.133333,BV1Y54y1f7j8,木子,赤小兔,3,0,3


In [104]:
N = len(df)
df_mini = df[N-183:N].sort_values('date', ascending=False)
df_mini

,bestof,date,team1,team2,owl,overgg,mvp,vod,comment1,comment2,winner_map1,winner_map2,winner_map3,winner_map4,winner_map5,winner_map6,winner_map7,score1,score2,maps
182,5,2022-08-20 21:00:00,val,phi,39059,16042,ZEST,https://www.youtube.com/watch?v=eyOqneIx2l0,Achilios,AVRL,1,2,2,1,2,,,2,3,5
181,5,2022-08-20 19:30:00,shd,hzs,39058,16041,LIP,https://www.youtube.com/watch?v=ABvOrx-Lx0g,Achilios,AVRL,1,1,1,,,,,3,0,3
180,5,2022-08-20 18:00:00,seo,cdh,39057,16040,Stalk3r,https://www.youtube.com/watch?v=7L_IoxIhjgU,Achilios,AVRL,1,1,1,,,,,3,0,3
179,5,2022-08-20 06:00:00,tor,dal,39211,16039,Edison,https://www.youtube.com/watch?v=O56NzdSUWlA,Jaws,Mr X,2,1,2,2,,,,1,3,4
178,5,2022-08-20 04:30:00,sfs,bos,39210,16038,Proper,https://www.youtube.com/watch?v=KjQ9SoOCpC8,Jaws,Mr X,1,1,1,,,,,3,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,5,2022-05-07 04:30:00,bos,van,38974,15907,Punk,,LemonKiwi,LEGDAY,1,2,1,2,1,,,3,2,5
3,5,2022-05-07 03:00:00,ldn,sfs,38975,15908,s9mm,,Jaws,Nekkra,2,2,2,,,,,0,3,3
2,5,2022-05-06 06:00:00,fla,atl,38973,15904,UltraViolet,,Jaws,Nekkra,1,2,2,2,,,,1,3,4
1,5,2022-05-06 04:30:00,sfs,par,38972,15903,FiNN,,Uber,Mr X,1,1,1,,,,,3,0,3


Switch team
```
\|opponent1=\{\{TeamOpponent\|(.+)\|score=\}\}\n\s+\|opponent2=\{\{TeamOpponent\|(.+)\|score=\}\}
|opponent1={{TeamOpponent|$2|score=}}\n    |opponent2={{TeamOpponent|$1|score=}}
```

Switch score
```
Map\|map=(.+)\|mode=(.+)\|score1=(.+)\|score2=(.+)\|
Map|map=$1|mode=$2|score1=$4|score2=$3|
```

**需要检查夏季锦标赛的队伍顺序**

2022-09-10 07:00:00	was	tor (目前好像就这个和视频内对应)

In [105]:
for i, (t1, t2, s1, s2, nm) in enumerate(zip(df_mini['team1'], df_mini['team2'], df_mini['score1'], df_mini['score2'], df_mini['maps'])):
    r = df_old.iloc[i]
    if set((dteam[t1], dteam[t2])) != set((r['team1'], r['team2'])):
        print(i)
    if s1 != r['score1']:
        print(i)
    if s2 != r['score2']:
        print(i)
    if nm != r['maps']:
        print(i)

In [106]:
dN = len(df) - len(df_old)
print(dN, len(df), len(df_old))
from copy import copy
df_total = copy(df)
df_total['name1'] = [""] * dN + list(df_old['name1'])
df_total['name2'] = [""] * dN + list(df_old['name2'])
df_total
# df_total.to_excel('owl_raw_keep_update_0910.xlsx')

49 232 183


##### 和以前从 liquipedia 对比

In [279]:
# df_old = pd.read_excel('owl_raw_keep_update_0910.xlsx', usecols=range(1, 23))
df_old = pd.read_excel('owl_raw_keep_update_0910v2.xlsx', usecols=range(1, 37))
# df_old
df_old[['date', 'team1', 'team2', 'score1', 'score2', 'maps', 'name1', 'name2']]

,date,team1,team2,score1,score2,maps,name1,name2
0,2022-09-10 07:00:00,was,tor,1,3,4,杨小龙,蛋壳
1,2022-09-10 05:35:00,van,fla,0,3,3,杨小龙,蛋壳
2,2022-09-10 03:45:00,sfs,hou,3,1,4,时光,桃叽
3,2022-09-10 02:00:00,dal,ldn,3,1,4,时光,桃叽
4,2022-09-09 19:30:00,seo,phi,3,2,5,九朵,老陈
...,...,...,...,...,...,...,...,...
227,2022-05-07 04:30:00,bos,van,3,2,5,九朵,Alan
228,2022-05-07 03:00:00,ldn,sfs,0,3,3,九朵,时光
229,2022-05-06 06:00:00,fla,atl,1,3,4,木子,赤小兔
230,2022-05-06 04:30:00,sfs,par,3,0,3,木子,赤小兔


In [280]:
# df_mini = df[:212].sort_values('date', ascending=False)
dN = len(df) - len(df_old)
df_mini = df[dN:len(df)]
df_mini[['date', 'team1', 'team2', 'mvp', 's1m1', 'score1', 'score2', 'maps']]

,date,team1,team2,mvp,s1m1,score1,score2,maps
228,2022-09-10 07:00:00,was,tor,ALTHOUGH,0,1,3,4
229,2022-09-10 05:35:00,van,fla,Checkmate,1,0,3,3
227,2022-09-10 03:45:00,sfs,hou,Proper,2,3,1,4
226,2022-09-10 02:00:00,dal,ldn,Sp9rk1e,2,3,1,4
231,2022-09-09 19:30:00,seo,phi,Stalk3r,2,3,2,5
...,...,...,...,...,...,...,...,...
4,2022-05-07 04:30:00,bos,van,Punk,2,3,2,5
3,2022-05-07 03:00:00,ldn,sfs,s9mm,0,0,3,3
2,2022-05-06 06:00:00,fla,atl,UltraViolet,2,1,3,4
1,2022-05-06 04:30:00,sfs,par,FiNN,2,3,0,3


In [281]:
for i, (t1, t2, s1, s2, nm) in enumerate(zip(df_mini['team1'], df_mini['team2'], df_mini['score1'], df_mini['score2'], df_mini['maps'])):
    r = df_old.iloc[i]
    # if set((dteam[t1], dteam[t2])) != set((r['team1'], r['team2'])):
    if set((t1, t2)) != set((r['team1'], r['team2'])):
        print(i)
    # if s1 != r['score1']:
    #     print(i)
    # if s2 != r['score2']:
    #     print(i)
    # if nm != r['maps']:
    #     print(i)

In [282]:
dN = len(df) - len(df_old)
print(dN, len(df), len(df_old))
from copy import copy
df_total = copy(df)
df_total['name1'] = [""] * dN + list(df_old['name1'])
df_total['name2'] = [""] * dN + list(df_old['name2'])
df_total
df_total.to_excel('owl_raw_keep_update_0910v3.xlsx')

2 234 232


### 分析夏季锦标赛以来数据 (包含小图比分)

In [312]:
# df = pd.read_excel('owl_raw_keep_update_0910v2.xlsx', usecols=range(1, 37))
df = pd.read_excel('owl_raw_keep_update_0910v3.xlsx', usecols=range(1, 37))
df = df[:72]    # 仅统计夏季锦标赛以来数据
# df

In [313]:
names = set(df['name1']) | set(df['name2'])
print(', '.join(list(names)))

娓娓, 阳小龙, 赤小兔, 2dm, 桃叽, 狂疯小逗, 时光, Youngjin, 木子, 老陈, 蛋壳, Alan, 九朵


In [314]:
# 需要统计解说的 3-0 比赛中小分的总和以及出现 0 的占比
lMatch, lMap, lMatch0, lMatch1, lMatch2, lMap0, lMap0InMatch0 = [], [], [], [], [], [], []     # 解说场次数量, 解说地图数, 3-0数量, 解说0的小图数, 3-0中小图为0的图数
for name in names:
    match, map, match0, match1, match2, map0, map0InMatch0 = 0, 0, 0, 0, 0, 0, 0
    for _, row in df.iterrows():
        if name == row['name1'] or name == row['name2']:
            match += 1
            m = row['maps']
            map += m
            for i in range(7):
                if row[f's1m{i+1}'] == 0 or row[f's2m{i+1}'] == 0:
                    map0 += 1
                    if m == 3:
                        map0InMatch0 += 1
                
            if m == 3:
                match0 += 1
            if m == 4:
                match1 += 1
            if m == 5:
                match2 += 1
    lMatch.append(match)
    lMap.append(map)
    lMatch0.append(match0)
    lMatch1.append(match1)
    lMatch2.append(match2)
    lMap0.append(map0)
    lMap0InMatch0.append(map0InMatch0)

lMatch, lMap, lMatch0, lMatch1, lMatch2, lMap0, lMap0InMatch0 = np.array(lMatch), np.array(lMap), np.array(lMatch0), np.array(lMatch1), np.array(lMatch2), np.array(lMap0), np.array(lMap0InMatch0)

In [315]:
df_summer_names = pd.DataFrame(list(zip(names,    lMatch,   lMap, lMap/lMatch,  lMatch0,  lMatch1,  lMatch2, lMatch0/lMatch, lMap0, lMap0/lMap, lMap0InMatch0, lMap0InMatch0 / (3*lMatch0 +1e-12))), 
                                columns=['names', 'match', 'map', 'map/match', 'match0', 'match1', 'match2', 'match0/match', 'map0', 'map0/map', 'Map0InMatch0', 'Map0InMatch0/map0'])
df_summer_names.sort_values(by='map/match', ascending=False)

,names,match,map,map/match,match0,match1,match2,match0/match,map0,map0/map,Map0InMatch0,Map0InMatch0/map0
2,赤小兔,2,10,5.000000,0,0,2,0.000000,1,0.100000,0,0.000000
7,Youngjin,6,26,4.333333,0,4,2,0.000000,7,0.269231,0,0.000000
11,Alan,18,73,4.055556,5,7,6,0.277778,17,0.232877,4,0.266667
1,阳小龙,12,47,3.916667,3,7,2,0.250000,15,0.319149,5,0.555556
4,桃叽,12,47,3.916667,5,3,4,0.416667,14,0.297872,6,0.400000
3,2dm,8,31,3.875000,3,3,2,0.375000,10,0.322581,3,0.333333
9,老陈,18,68,3.777778,9,4,5,0.500000,19,0.279412,7,0.259259
0,娓娓,8,30,3.750000,3,4,1,0.375000,7,0.233333,2,0.222222
5,狂疯小逗,8,29,3.625000,5,1,2,0.625000,11,0.379310,7,0.466667
8,木子,8,29,3.625000,4,3,1,0.500000,7,0.241379,2,0.166667


In [316]:
# plot
import plotly.express as px

In [317]:
fig = px.line(df_summer_names.sort_values(by='map/match', ascending=False), 
              x="names", y="map/match", 
              labels={'names':'解说', 'map/match':'平均地图数'},
              text="map/match")
fig.update_traces(textposition="top right", texttemplate="%{y:.2f}")
fig.show()

In [318]:
names, maps, rates = [], [], []
for _, row in df_summer_names.sort_values(by='match0/match', ascending=False).iterrows():
    name, zero, one, two = row['names'], row['match0'], row['match1'], row['match2']
    total = zero + one + two
    for map, i in zip([3, 4, 5], [zero, one, two]):
        names.append(name)
        maps.append(map)
        rates.append(i / total)
df_cum = pd.DataFrame(list(zip(names, maps, rates)), 
                        columns=['names', 'maps', 'rate'])

fig = px.histogram(df_cum,
                    x="names", y="rate",
                    color='maps', # barmode='group',
                    # histfunc='avg',
                    labels={'names':'解说', 'rate':'解说比赛地图数占比'},
                    text_auto=True,
                    height=400).update_layout(yaxis_title='解说比赛地图数占比')
fig.update_traces(texttemplate="%{y:.2f}")
fig.show()

In [331]:
import plotly.graph_objects as go
animals=['giraffes', 'orangutans', 'monkeys']
df_plot = df_summer_names.sort_values(by='Map0InMatch0', ascending=False)

fig = go.Figure(data=[
    go.Bar(name='零封地图数', x=df_plot['names'], y=df_plot['map0'], text=df_plot['map0']),
    go.Bar(name='3-0中的零封地图数', x=df_plot['names'], y=df_plot['Map0InMatch0'], text=df_plot['Map0InMatch0'])
])
# Change the bar mode
fig.update_layout(barmode='group', yaxis_title='地图数')
fig.show()

### 分析

In [320]:
df = pd.read_excel('owl_raw_keep_update_0910v3.xlsx', usecols=range(1, 37))
df[0:1]

,bestof,date,team1,team2,owl,overgg,mvp,vod,comment1,comment2,...,s2m6,wm6,s1m7,s2m7,wm7,score1,score2,maps,name1,name2
0,5,2022-09-10 19:25:00,shd,seo,39283.0,16474.0,LIP,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3,0,3,木子,时光


In [321]:
names = set(df['name1']) | set(df['name2'])
print(', '.join(list(names)))

小霜, Roy, 娓娓, 阳小龙, 赤小兔, 2dm, 桃叽, 花猫, 狂疯小逗, 银河, 时光, Youngjin, 木子, 老陈, 蛋壳, Alan, 九朵


In [322]:
all, total, maps, zero, mzeros, one, mones, two, mtwos = [], [], [], [], [], [], [], [], []
for name in names:
    all_length, total_match, total_map, zero_match, one_match, two_match = 0, 0, 0, 0, 0, 0
    max0, czero = 0, 0 # 连续的不加班天数
    max1, cone  = 0, 0 # 连续三一重工
    max2, ctwo  = 0, 0 # 连续加班!
    flag0, flag1, flag2 = False, False, False
    for _, row in df.iterrows():
        if name == row['name1'] or name == row['name2']:
            # all_length += row['length']
            total_match += 1
            # s1, s2, m = row['score1'], row['score2'], row['maps']
            m = row['maps']
            total_map += m
            if m == 3:
                zero_match += 1
                czero = czero + 1 if flag0 else 1
                flag0, max0 = True, max(czero, max0)
                flag1, cone , max1 = False, 0, max(cone , max1)
                flag2, ctwo , max2 = False, 0, max(ctwo , max2)
            elif m == 4:
                one_match += 1
                flag0, czero, max0 = False, 0, max(czero, max0)
                cone  = cone + 1 if flag1 else 1
                flag1, max1 = True, max(cone , max1)
                flag2, ctwo , max2 = False, 0, max(ctwo , max2)
            elif m == 5:
                two_match += 1
                flag0, czero, max0 = False, 0, max(czero, max0)
                flag1, czero, max1 = False, 0, max(czero, max1)
                ctwo   = ctwo + 1 if flag2 else 1
                flag2, max2 = True, max(ctwo , max2)
    # all.append(all_length)
    total.append(total_match)
    zero.append(zero_match)
    one.append(one_match)
    two.append(two_match)
    mzeros.append(max0)
    mones.append(max1)
    mtwos.append(max2)
    maps.append(total_map)

total, maps, zero, one, two = np.array(total), np.array(maps), np.array(zero), np.array(one), np.array(two)

In [323]:
df_names = pd.DataFrame(list(zip(names, total, maps, maps/total, zero, one, two, mzeros, mones, mtwos, zero/total, one/total, two/total)), 
                        columns=['names', 'total', 'maps', 'ave', 'zero', 'one', 'two', 'max0', 'max1', 'max2', 'zero_rate', 'one_rate', 'two_rate'])

# df_names = df_names[df_names['all'] > 1000]   # 排除时间较短的
# df_names = df_names[df_names['total'] < 10]   # 排除时间较短的

# df_names.sort_values(by='zero_rate', ascending=False)
# df_names.sort_values(by='one_rate', ascending=False)
df_names.sort_values(by='two_rate', ascending=False)

# df_names.sort_values(by='max0', ascending=False)
# df_names.sort_values(by='max2', ascending=False)

# df_names

,names,total,maps,ave,zero,one,two,max0,max1,max2,zero_rate,one_rate,two_rate
9,银河,6,25,4.166667,2,1,3,1,1,3,0.333333,0.166667,0.500000
4,赤小兔,18,75,4.166667,4,7,7,1,2,3,0.222222,0.388889,0.388889
11,Youngjin,22,89,4.045455,6,9,7,3,3,2,0.272727,0.409091,0.318182
14,蛋壳,10,37,3.700000,6,1,3,3,1,2,0.600000,0.100000,0.300000
13,老陈,28,106,3.785714,14,6,8,3,1,2,0.500000,0.214286,0.285714
1,Roy,18,69,3.833333,8,5,5,4,2,2,0.444444,0.277778,0.277778
15,Alan,71,277,3.901408,26,26,19,4,3,2,0.366197,0.366197,0.267606
8,狂疯小逗,8,29,3.625000,5,1,2,3,1,1,0.625000,0.125000,0.250000
16,九朵,60,233,3.883333,22,23,15,7,4,2,0.366667,0.383333,0.250000
10,时光,60,230,3.833333,24,23,12,5,5,2,0.400000,0.383333,0.200000


In [173]:
# plot
import plotly.express as px

In [324]:
names, maps, rates = [], [], []
for _, row in df_names.sort_values(by='one_rate', ascending=False).iterrows():
    name, zero, one, two = row['names'], row['zero'], row['one'], row['two']
    total = zero + one + two
    for map, i in zip([3, 4, 5], [zero, one, two]):
        names.append(name)
        maps.append(map)
        rates.append(i / total)
df_cum = pd.DataFrame(list(zip(names, maps, rates)), 
                        columns=['names', 'maps', 'rate'])

fig = px.histogram(df_cum,
                    x="names", y="rate",
                    color='maps', # barmode='group',
                    # histfunc='avg',
                    labels={'names':'解说', 'rate':'解说比赛地图数占比'},
                    height=400)
fig.show()

In [329]:
fig = px.histogram(df_names[df_names['total'] >= 10].sort_values(by='two_rate', ascending=False),
                    x="names", y="two_rate",
                    color='max2', # barmode='group',
                    histfunc='avg',
                    labels={'names':'解说', 'two_rate':'解说到第 5 张地图占比'},
                    text_auto=True,
                    height=400).update_layout(yaxis_title='解说到第 5 张地图占比')
fig.update_traces(texttemplate="%{y:.2f}")
fig.show()

In [330]:
fig = px.histogram(df_names.sort_values(by='two_rate', ascending=False),
                    x="names", y="two_rate",
                    color='max2', # barmode='group',
                    histfunc='avg',
                    labels={'names':'解说', 'two_rate':'解说到第 5 张地图占比'},
                    text_auto=True,
                    height=400).update_layout(yaxis_title='解说到第 5 张地图占比')
fig.update_traces(texttemplate="%{y:.2f}")
fig.show()

In [177]:
fig = px.histogram(df_names[df_names['total'] >= 10].sort_values(by='one_rate', ascending=False),
                    x="names", y="one_rate",
                    color='max1', # barmode='group',
                    histfunc='avg',
                    labels={'names':'解说', 'one_rate':'解说比赛三一重工占比'},
                    height=400)
fig.show()

In [179]:
fig = px.histogram(df_names.sort_values(by='one_rate', ascending=False),
                    x="names", y="one_rate",
                    color='max1', # barmode='group',
                    histfunc='avg',
                    labels={'names':'解说', 'one_rate':'解说比赛共 4 张地图占比'},
                    height=400)
fig.show()

In [326]:
fig = px.histogram(df_names.sort_values(by='max0', ascending=False),
                    x="names", y="max0",
                    histfunc='avg',
                    labels={'names':'解说', 'max0':'连续不加班次数'}, text_auto=True,
                    height=400).update_layout(yaxis_title='连续不加班次数')
fig.show()

In [327]:
fig = px.line(df_names.sort_values(by='zero_rate', ascending=False), 
                x="names", y="zero_rate", 
                labels={'names':'解说', 'zero_rate':'3-0占比'},
                text="ave")
fig.update_traces(textposition="top right", texttemplate="%{y:.2f}")
fig.show()

In [328]:
fig = px.line(df_names.sort_values(by='ave', ascending=False), 
                x="names", y="ave", 
                labels={'names':'解说', 'ave':'平均地图数'},
                text="ave")
fig.update_traces(textposition="top right", texttemplate="%{y:.2f}")
fig.show()

In [227]:
# 站位决定成败
names = set(df['name1']) | set(df['name2'])
print(', '.join(list(names)))

total, win = [], []
for name in names:
    t, w = 0, 0
    for _, row in df.iterrows():
        s1, s2 = row['score1'], row['score2']
        cond1 = (name == row['name1']) and (s1 > s2)
        cond2 = (name == row['name2']) and (s1 < s2)
        if name in [row['name1'], row['name2']]:
            t += 1
        if cond1 or cond2:
            w += 1
    total.append(t)
    win.append(w)

total, win = np.array(total), np.array(win)

银河, 杨小龙, 花猫, 赤小兔, 狂疯小逗, 蛋壳, 九朵, 桃叽, Alan, Roy, 小霜, Youngjin, 木子, 老陈, 2dm, 时光, 娓娓


In [228]:
df_names = pd.DataFrame(list(zip(names, total, win, win/total)), 
                        columns=['names', 'total', 'win', 'win_rate'])
df_names.sort_values(by='win_rate', ascending=False)

,names,total,win,win_rate
1,杨小龙,16,11,0.687500
10,小霜,6,4,0.666667
11,Youngjin,22,13,0.590909
6,九朵,56,32,0.571429
3,赤小兔,16,9,0.562500
13,老陈,22,12,0.545455
7,桃叽,28,15,0.535714
15,时光,52,27,0.519231
16,娓娓,14,7,0.500000
4,狂疯小逗,4,2,0.500000
